In [1]:
import pandas as pd
import numpy as np
X_train = pd.read_csv("data/x_train.txt",header=None,sep=" ")
y_train = pd.read_csv("data/y_train.txt",header=None,sep=" ")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import f_classif
import matplotlib.pyplot as plt
import seaborn as sns
f_values, p_values = f_classif(X_train, y_train)


anova_df = pd.DataFrame({
    'Feature': range(len(p_values)),  
    'p_values': p_values
})
anova_df.sort_values('p_values',ascending=False)

C:\Users\Michał\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Feature,p_values
111,111,9.997676e-01
296,296,9.991506e-01
448,448,9.977965e-01
53,53,9.975327e-01
466,466,9.944021e-01
...,...,...
4,4,7.587568e-117
7,7,1.062914e-119
6,6,1.980223e-134
3,3,2.693812e-135


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score


def evaluate_rfe_with_custom_score(X, y, step=1, min_features=1, max_features=15, cv=5):
    y= np.array(y)
    num_target=1000
    n_splits = 5
    for n_features in range(min_features, max_features + 1, step):
        print(f"Testing on {n_features} features")
        model = XGBClassifier()
        selector = RFE(estimator=model, n_features_to_select=n_features, step=0.25, verbose=0)
        selector.fit(X, y)
        X_selected = selector.transform(X)
        X_selected = np.array(X_selected)
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        probas = np.zeros(X.shape[0])  

        for train_idx, test_idx in cv.split(X_selected, y):
            X_train, X_val = X_selected[train_idx,], X_selected[test_idx,]
            y_train, y_val = y[train_idx], y[test_idx]
    
            model = XGBClassifier(
            )
            
            model.fit(
                X_train, y_train,
                verbose=False
            )
            
            probas[test_idx] = model.predict_proba(X_val)[:,1]

        sorted_probas = np.sort(probas)[::-1]  
        threshold = sorted_probas[num_target - 1]

        y_pred = (probas >= threshold).astype(int)
        gain = 10*precision_score(y, y_pred)*num_target
        print (gain - 200*n_features)

results = evaluate_rfe_with_custom_score(X_train, y_train)



Testing on 1 features
7018.344965104686
Testing on 2 features
6559.999999999999
Testing on 3 features
6379.999999999999
Testing on 4 features
6330.0
Testing on 5 features
6199.999999999999
Testing on 6 features
6010.0
Testing on 7 features
5780.0
Testing on 8 features
5460.0
Testing on 9 features
5460.0
Testing on 10 features
5330.0
Testing on 11 features
5100.0
Testing on 12 features
4900.0
Testing on 13 features
4660.0
Testing on 14 features
4429.999999999999
Testing on 15 features
4260.0
